In [1]:
import os
import csv
import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim.nets import inception
from scipy.misc import imread
from scipy.misc import imresize
from cleverhans.attacks import MomentumIterativeMethod
from cleverhans.attacks import Model
from PIL import Image
slim = tf.contrib.slim
# tf.flags.DEFINE_string('f','kernel', '')
tf.flags.DEFINE_string(
    'checkpoint_path', '../official_data/model/inception_v1/inception_v1.ckpt', 'Path to checkpoint for inception network.')
tf.flags.DEFINE_string(
    'input_dir', '../official_data/dev_data/', 'Input directory with images.')
tf.flags.DEFINE_string(
    'output_dir', '', 'Output directory with images.')
tf.flags.DEFINE_integer(
    'image_width', 224, 'Width of each input images.')
tf.flags.DEFINE_integer(
    'image_height', 224, 'Height of each input images.')
tf.flags.DEFINE_integer(
    'batch_size', 110, 'How many images process at one time.')
tf.flags.DEFINE_integer(
    'num_classes', 110, 'Number of Classes')
FLAGS = tf.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
# from IJCAI19.module.utils import * 
def load_images(input_dir, batch_shape):
    images = np.zeros(batch_shape)
    labels = np.zeros(batch_shape[0], dtype=np.int32)
    filenames = []
    idx = 0
    batch_size = batch_shape[0]
    with open(os.path.join(input_dir, 'dev.csv')) as f:
        reader = csv.DictReader(f)
        for row in reader:
            filepath = os.path.join(input_dir, row['filename'])
            with open(filepath, 'rb') as f:
                raw_image = imread(f, mode='RGB').astype(np.float)
                image = imresize(raw_image, [FLAGS.image_height, FLAGS.image_width]) / 255.0
            # Images for inception classifier are normalized to be in [-1, 1] interval.
            images[idx, :, :, :] = image * 2.0 - 1.0
            labels[idx] = int(row['targetedLabel'])
            filenames.append(os.path.basename(filepath))
            idx += 1
            if idx == batch_size:
                yield filenames, images, labels
                filenames = []
                images = np.zeros(batch_shape)
                labels = np.zeros(batch_shape[0], dtype=np.int32)
                idx = 0
        if idx > 0:
            yield filenames, images, labels


def save_images(images, filenames, output_dir):
    for i, filename in enumerate(filenames):
        # Images for inception classifier are normalized to be in [-1, 1] interval,
        # so rescale them back to [0, 1].
        with open(os.path.join(output_dir, filename), 'w') as f:
            img = (((images[i, :, :, :] + 1.0) * 0.5) * 255.0).astype(np.uint8)
            # resize back to [299, 299]
            r_img = imresize(img, [299, 299])
            Image.fromarray(r_img).save(f, format='PNG')

In [16]:

class InceptionModel(Model):
    """Model class for CleverHans library."""
    def __init__(self, x_input, nb_classes):
        super(InceptionModel, self).__init__(nb_classes=nb_classes,
                                             needs_dummy_fprop=True)
        self.built = False
        self.model_endpoints = None
        with slim.arg_scope(inception.inception_v1_arg_scope()):
            _, self.model_endpoints = inception.inception_v1(
                x_input, num_classes=self.nb_classes, is_training=False,
                reuse=False)

    def __call__(self, x_input, return_logits=False):
        """Constructs model and return probabilities for given input."""
        reuse = True if self.built else None
#         with slim.arg_scope(inception.inception_v1_arg_scope()):
#             _, end_points = inception.inception_v1(
#                 x_input, num_classes=self.nb_classes, is_training=False,
#                 reuse=reuse)
        end_points = self.model_endpoints
        self.built = True
        self.logits = end_points['Logits']
        # Strip off the extra reshape op at the output
        self.probs = end_points['Predictions'].op.inputs[0]
        if return_logits:
            return self.logits
        else:
            return self.probs

    def get_logits(self, x_input):
        return self(x_input, return_logits=True)

    def get_probs(self, x_input):
        return self(x_input)


def main(_):
    """Run the sample attack"""
    batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
    nb_classes = FLAGS.num_classes
    tf.logging.set_verbosity(tf.logging.INFO)

    with tf.Graph().as_default():
        # Prepare graph
        x_input = tf.placeholder(tf.float32, shape=batch_shape)
        target_class_input = tf.placeholder(tf.int32, shape=[FLAGS.batch_size])
        one_hot_target_class = tf.one_hot(target_class_input, nb_classes)
        # Run computation
        with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
            model = InceptionModel(x_input, nb_classes)
            mim = MomentumIterativeMethod(model, sess=sess)
            attack_params = {"eps": 32.0 / 255.0, "eps_iter": 0.01, "clip_min": -1.0, "clip_max": 1.0, \
                             "nb_iter": 2, "decay_factor": 1.0, "y_target": one_hot_target_class}
#             x_adv = mim.generate(x_input, **attack_params)
#             saver = tf.train.Saver(slim.get_model_variables())
#             saver.restore(sess, FLAGS.checkpoint_path)
#             for filenames, images, tlabels in load_images(FLAGS.input_dir, batch_shape):
#                 adv_images = sess.run(x_adv,
#                                       feed_dict={x_input: images, target_class_input: tlabels})
#                 save_images(adv_images, filenames, FLAGS.output_dir)
                
            


In [26]:
from cleverhans.compat import reduce_sum, reduce_mean, softmax_cross_entropy_with_logits
from IJCAI19.module.gs_mim import GradSmoothMomentumIterativeMethod
batch_shape = [None, FLAGS.image_height, FLAGS.image_width, 3]
nb_classes = FLAGS.num_classes
tf.logging.set_verbosity(tf.logging.INFO)

with tf.Graph().as_default():
    # Prepare graph
    x_input = tf.placeholder(tf.float32, shape=batch_shape)
    target_class_input = tf.placeholder(tf.int32, shape=[FLAGS.batch_size])
    one_hot_target_class = tf.one_hot(target_class_input, nb_classes)
    # Run computation
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        model = InceptionModel(x_input, nb_classes)
        mim = GradSmoothMomentumIterativeMethod(model, sess=sess)
        attack_params = {"eps": 32.0 / 255.0, "eps_iter": 0.01, "clip_min": -1.0, "clip_max": 1.0, \
                         "nb_iter": 2, "decay_factor": 1.0, "y_target": one_hot_target_class}
#         x_adv = mim.generate(x_input, **attack_params)
#         saver = tf.train.Saver(slim.get_model_variables())
#         saver.restore(sess, FLAGS.checkpoint_path)
#         for filenames, images, tlabels in load_images(FLAGS.input_dir, batch_shape):
#             adv_images = sess.run(x_adv,
#                                   feed_dict={x_input: images, target_class_input: tlabels})
#             save_images(adv_images, filenames, FLAGS.output_dir)
        def cond(i, _, __):
          return tf.less(i, 2)
        def body(i, ax, m):
            y =target_class_input
            logits = model.get_logits(ax)
            loss = logits
#             loss = softmax_cross_entropy_with_logits(labels=y, logits=logits)
            grad, = tf.gradients(loss, ax)
            print(grad)
            return i + 1, ax, m
        adv_x = x_input
        _, adv_x, _ = tf.while_loop(
            cond, body, (tf.zeros([]), x_input, 1), back_prop=True,
            maximum_iterations=2)

#         y =target_class_input
#         logits = model.get_logits(x_input)
#         loss = softmax_cross_entropy_with_logits(labels=y, logits=logits)
#         grad, = tf.gradients(loss, x_input)

None


In [19]:
adv_x

<tf.Tensor 'while/Exit_2:0' shape=(?, 224, 224, 3) dtype=float32>